# 02-experiment-tracking: Homework

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management.

In [11]:
import os
import mlflow
import subprocess
from mlflow.tracking import MlflowClient

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [12]:
mlflow.__version__

'2.12.2'

## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [10]:
!python ./scripts/preprocess_data.py --raw_data_path ./data/input --dest_path ./data/output
!ls -1 ./data/output | wc -l

       4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

In [14]:
# Start the MLflow UI at http://127.0.0.1:5000 in the background
MLFLOW_UI_COMMAND = [
    "mlflow", "ui", 
    "--backend-store-uri", "sqlite:///mlflow.db", 
    "--default-artifact-root", "./mlruns",
    "--port", "5000",
]
mlflow_ui_process = subprocess.Popen(
    MLFLOW_UI_COMMAND,
    cwd=os.getcwd(),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
print(f"Started MLflow UI with PID: {mlflow_ui_process.pid}")


Started MLflow UI with PID: 66748


In [7]:
!python ./scripts/train.py

2024/05/27 12:19:03 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/27 12:19:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/05/27 12:19:04 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 12:19:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/begonasanchez/Library/CloudStorage/OneDrive-FrontiersMediaSA/Documents/Other/MLE/mlops-zoomcamp/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/Users/begonasanchez/Library/CloudStorage/OneDrive-FrontiersMediaSA/Documents/Other/MLE/mlops-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: Future

In [15]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()
experiment = client.get_experiment_by_name("Default")
runs = client.search_runs(experiment_ids=[experiment.experiment_id], order_by=["start_time desc"], max_results=1)
last_run = runs[0]
last_run_min_samples_split = last_run.data.params.get("min_samples_split")
print(f"min_samples_split from the last run: {last_run_min_samples_split}")

min_samples_split from the last run: 2


## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`



In [16]:
# Start the MLflow server at http://127.0.0.1:5001 in the background
MLFLOW_SERVER_COMMAND = [
    "mlflow", "server", 
    "--backend-store-uri", "sqlite:///backend.db", 
    "--default-artifact-root", "./artifacts",
    "--port", "5001",
]
mlflow_server_process = subprocess.Popen(
    MLFLOW_SERVER_COMMAND,
    cwd=os.getcwd(),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
print(f"Started MLflow server with PID: {mlflow_server_process.pid}")

Started MLflow server with PID: 66899


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

In [18]:
!python ./scripts/hpo.py

  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/Users/begonasanchez/Library/CloudStorage/OneDrive-FrontiersMediaSA/Documents/Other/MLE/mlops-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:05<01:12,  5.16s/trial, best loss: 5.370086069268862]/Users/begonasanchez/Library/CloudStorage/OneDrive-FrontiersMediaSA/Documents/Other/MLE/mlops-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:05<00:30,  2.34s/trial, best loss: 5.370086069268862]/Users/begonasanchez/Library/CloudStorage/

In [30]:
mlflow.set_tracking_uri("http://127.0.0.1:5001")
client = MlflowClient()
experiment = client.get_experiment_by_name("random-forest-hyperopt")

runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.test_rmse ASC"],
    max_results=1,
)
lowest_rmse = runs[0].data.metrics.get("rmse")
print(f"Best validation RMSE: {lowest_rmse}")

Best validation RMSE: 5.335419588556921


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [41]:
!python ./scripts/register_model.py

2024/05/27 19:27:35 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/27 19:27:38 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/27 19:27:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/begonasanchez/Library/CloudStorage/OneDrive-FrontiersMediaSA/Documents/Other/MLE/mlops-zoomcamp/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/Users/begonasanchez/Library/CloudStorage/OneDrive-FrontiersMediaSA/Documents/Other/MLE/mlops-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the ro

In [47]:
mlflow.set_tracking_uri("http://127.0.0.1:5001")
client = MlflowClient()
model_name = "random-forest-regressor-best"
latest_version_info = client.get_latest_versions(name=model_name, stages=["None", "Staging", "Production"])[0]
run_id = latest_version_info.run_id
run = client.get_run(run_id)
rmse = run.data.metrics.get("test_rmse")
print(f"Test RMSE of the best model (Run ID: {run_id}): {rmse}")

Test RMSE of the best model (Run ID: f7f3734a3153447cbf51bed91d8ba468): 5.567408012462019


/var/folders/y2/hm445s210vs040mn_6ytg2jh0000gp/T/ipykernel_87646/1609695606.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_version_info = client.get_latest_versions(name=model_name, stages=["None", "Staging", "Production"])[0]
